## DDPG Portfolio Management

In [141]:
import numpy as np
from allocation_env import AssetAllocationEnv
from ddpg_agent import DDPGAgent
import pickle
import pandas as pd
import tensorflow as tf
from rl_interface import RLInterface
tf.get_logger().setLevel('ERROR')
from market_data import MarketData
import datetime as dt
from keras.layers import Dense, Flatten
from keras.activations import relu
from keras.initializers import glorot_uniform
from transformer_encoder import EncoderLayer

#### Number of stocks in the portfolio

In [142]:
num_stocks = 10
start_date = '2013-01-03'
end_date = '2024-02-01'

#### Import locally stored data for stock prices, macro variables, and risk-free rate

In [143]:
with open('data/top_10_stocks_2013-01-03_to_2024-02-01_as_of_02-10-2024.pkl', 'rb') as file:
    price_dict = pickle.load(file)  # dictionary of dataframes of the form {ticker: dataframe of candles}

In [144]:
price_dict['AAPL']

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2013-01-03,19.567142,19.631071,19.321428,19.360714,16.536327,352965200
2013-01-04,19.177500,19.236786,18.779642,18.821428,16.075716,594333600
2013-01-07,18.642857,18.903570,18.400000,18.710714,15.981145,484156400
2013-01-08,18.900356,18.996071,18.616072,18.761070,16.024164,458707200
2013-01-09,18.660713,18.750357,18.428213,18.467857,15.773727,407604400
...,...,...,...,...,...,...
2024-01-25,195.220001,196.270004,193.110001,194.169998,193.922546,54822100
2024-01-26,194.270004,194.759995,191.940002,192.419998,192.174774,44594000
2024-01-29,192.009995,192.199997,189.580002,191.729996,191.485657,47145600


In [145]:
with open('data/macro_factors_2013-01-03_to_2024-02-01.pkl', 'rb') as file:
    macro = pickle.load(file)
macro

,5 Yr BE Inf,5 Yr Fwd Inf Exp,Fed Funds Rate,10-2 Yr Tr Yld,VIX
Date,,,,,
2013-01-03,2.11,2.81,0.17,1.65,14.56
2013-01-04,2.13,2.83,0.16,1.66,13.83
2013-01-07,2.16,2.88,0.16,1.65,13.79
2013-01-08,2.15,2.83,0.15,1.64,13.62
2013-01-09,2.18,2.86,0.14,1.64,13.81
...,...,...,...,...,...
2024-01-25,2.24,2.34,5.33,-0.14,13.45
2024-01-26,2.26,2.34,5.33,-0.19,13.26
2024-01-29,2.26,2.34,5.33,-0.21,13.60


In [146]:
with open('data/3_mo_treasury_2013-01-03_to_2024-02-01.pkl', 'rb') as file:
    rf = pickle.load(file)

In [147]:
rf

Date
2013-01-03    0.0008
2013-01-04    0.0007
2013-01-07    0.0007
2013-01-08    0.0007
2013-01-09    0.0006
               ...  
2024-01-25    0.0544
2024-01-26    0.0544
2024-01-29    0.0542
2024-01-30    0.0542
2024-01-31    0.0542
Name: 3 Mo Tr Yld, Length: 2788, dtype: float64

#### Fetch list of S&P ticker and weights (Use of of the Options Below: Cap or Equal Weighting)

##### *Cap Weighting*

In [148]:
# with open('data/sp500_weights_as_of_02-10-2024.pkl', 'rb') as file:
#     sp_df = pickle.load(file)
# display(sp_df)
# sp_weights = sp_df['Weight'].iloc[:num_stocks]/sum(sp_df['Weight'].iloc[:num_stocks])
# sp_weights = sp_weights.to_numpy()
# sp_weights_array = np.repeat(sp_weights[np.newaxis, :], len(price_dict[list(price_dict.keys())[0]]), axis=0)

##### *Equal Weighting*

In [149]:
sp_weights = np.ones(num_stocks)/num_stocks
sp_weights_array = np.repeat(sp_weights[np.newaxis, :], len(price_dict[list(price_dict.keys())[0]]), axis=0)

#### Initalize MarketData object for input into RL environment

In [150]:
market_data = MarketData(price_dict, sp_weights_array, rf, macro, scaler='minmax', flatten=True, train_prop=0.7)

#### Environment Instantiation

In [151]:
# Create 2 environments: one for training and one for testing which access mutually exclusive (and consecutive) time series from market_data
train_env = AssetAllocationEnv(market_data=market_data, lookback=30, holding_period=5, 
                               window_size=150, rebalance=True, reward_type='alpha', train=True)
test_env = AssetAllocationEnv(market_data=market_data, lookback=30, holding_period=5, 
                              window_size=200, rebalance=True, reward_type='alpha', train=False)

lr_actor = 0.005
lr_critic = 0.005
tau = 0.005
gamma = 0.99
sigma = 1

buffer_capacity = 100000
batch_size = 256

dir = 'models/encoder_model'
chkpt_dir = dir + '/network_weights'
history_dir = dir + '/history'

#### Neural Network Architectures

In [152]:
# Middle layers of the network (Input and output layers are defined in class for Actor and Critic models) 
actor_layers = [
    EncoderLayer(input_shape=train_env.obs_shape, head_size=5, num_heads=3, gru_units=64,
    dense_units=512, dropout=0.1), 
    EncoderLayer(input_shape=train_env.obs_shape, head_size=5, num_heads=3, gru_units=64,
    dense_units=512, dropout=0.1),
    Flatten(),
    Dense(512, activation=relu, kernel_initializer=glorot_uniform()),  
    Dense(512, activation=relu, kernel_initializer=glorot_uniform())  
]

critic_layers = [
    Dense(512, activation=relu, kernel_initializer=glorot_uniform()), 
    Dense(512, activation=relu, kernel_initializer=glorot_uniform()) 
]

#### GPU Distribution Strategy *(use if  multiple GPUs are available)*

In [153]:
strategy = tf.distribute.MirroredStrategy()

#### Agent Instantiation

In [154]:
agent = DDPGAgent(env=train_env, actor_layers=actor_layers, critic_layers=critic_layers, lr_actor=lr_actor, 
                  lr_critic=lr_critic, chkpt_dir=chkpt_dir, tau=tau, gamma=gamma, sigma=sigma, 
                  buffer_capacity=buffer_capacity, batch_size=batch_size, strategy=strategy)

In [155]:
print(agent.actor.summary())

Model: "actor_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_27 (InputLayer)       [(None, 30, 65)]          0         
                                                                 
 encoder_layer_12 (EncoderL  multiple                  29422     
 ayer)                                                           
                                                                 
 encoder_layer_13 (EncoderL  multiple                  29165     
 ayer)                                                           
                                                                 
 flatten_44 (Flatten)        multiple                  0         
                                                                 
 dense_48 (Dense)            multiple                  983552    
                                                                 
 dense_49 (Dense)            multiple                  262

In [156]:
print(agent.critic.summary())

Model: "critic_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_45 (Flatten)        multiple                  0         
                                                                 
 flatten_46 (Flatten)        multiple                  0         
                                                                 
 dense_50 (Dense)            multiple                  1004032   
                                                                 
 dense_51 (Dense)            multiple                  262656    
                                                                 
 dense_53 (Dense)            multiple                  513       
                                                                 
Total params: 1267201 (4.83 MB)
Trainable params: 1267201 (4.83 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [157]:
interface = RLInterface(train_env=train_env, test_env=test_env, agent=agent)

## Training

In [158]:
interface.load_models()

..... Loading Models .....
..... Models loaded successfully! .....


In [164]:
train_history = interface.train_agent(num_episodes=100, verbose=1, seed=None, advanced_stats=True) # If NaN appear in scores, data is not cleaned properly

Training progress:   0%|          | 0/100 [00:00<?, ?it/s]

Episode: 1/100, Score: -0.0151, 10 epi avg.: -0.0151, All-time avg.: -0.0151
Episode: 2/100, Score: -0.0043, 10 epi avg.: -0.0097, All-time avg.: -0.0097
Episode: 3/100, Score: -0.0380, 10 epi avg.: -0.0191, All-time avg.: -0.0191
Episode: 4/100, Score: 0.0214, 10 epi avg.: -0.0090, All-time avg.: -0.0090
Episode: 5/100, Score: 0.0067, 10 epi avg.: -0.0059, All-time avg.: -0.0059
Episode: 6/100, Score: -0.0402, 10 epi avg.: -0.0116, All-time avg.: -0.0116
Episode: 7/100, Score: -0.0108, 10 epi avg.: -0.0115, All-time avg.: -0.0115
Episode: 8/100, Score: -0.0141, 10 epi avg.: -0.0118, All-time avg.: -0.0118
Episode: 9/100, Score: -0.0077, 10 epi avg.: -0.0113, All-time avg.: -0.0113
Episode: 10/100, Score: 0.0135, 10 epi avg.: -0.0089, All-time avg.: -0.0089
Episode: 11/100, Score: -0.0141, 10 epi avg.: -0.0093, All-time avg.: -0.0093
Episode: 12/100, Score: 0.0054, 10 epi avg.: -0.0081, All-time avg.: -0.0081
Episode: 13/100, Score: 0.0026, 10 epi avg.: -0.0073, All-time avg.: -0.0073


#### Plot Learning

In [165]:
train_scores = train_history['avg_epi_score_history']
interface.plot_learning(train_scores, rolling_period=10)

#### Render Last Episode

In [161]:
train_env.render(plot_benchmark=True, plot_actions=True, plot_relative=True, plot_returns=True, plot_individual=True, plot_contribution=True)

#### Last Episode Performance

In [162]:
print('Agent Vol (Ann.): {:.3f}%'.format(np.std(train_env.portfolio_returns)*100*np.sqrt(252)))
print('Benchmark Vol (Ann.): {:.3f}%'.format(np.std(train_env.benchmark_returns)*100*np.sqrt(252)))
print('Correlation: %.3f' % np.corrcoef(np.squeeze(train_env.portfolio_returns), np.squeeze(train_env.benchmark_returns))[0,1])
beta = np.cov(np.squeeze(train_env.portfolio_returns), np.squeeze(train_env.benchmark_returns))[0,1] / np.var(train_env.benchmark_returns)
print('Beta: %.3f' % beta)
print('Excess Return: {:.3f}%'.format((np.cumprod(1+train_env.portfolio_returns)[-1] - np.cumprod(1+train_env.benchmark_returns)[-1])*100))
print('Episode Alpha (rf=0%): {:.3f}%'.format((np.cumprod(1+train_env.portfolio_returns)[-1] - beta*(np.cumprod(1+train_env.benchmark_returns)[-1]))*100))

Agent Vol (Ann.): 20.172%
Benchmark Vol (Ann.): 15.026%
Correlation: 0.896
Beta: 1.213
Excess Return: 17.607%
Episode Alpha (rf=0%): -6.785%


#### Training Sample Performance Statistics

In [163]:
interface.print_advanced_stats(train_history)

Avg. Reward: 0.000
Avg. Volatility (Agent, Ann.): 18.886%
Avg. Risk Utilization (% of benchmark): 109.002%
Avg. Beta: 1.044
Avg. Correlation: 0.950
Avg. Return (Epi.): 115.906%
Avg. Excess Return (Epi.): 3.129%
Avg. Alpha (Ann.): 0.020%


#### Save model checkpoint to local directory

In [35]:
interface.save_models()

..... Saving models .....
..... Models saved successfully! .....


## Testing

In [44]:
test_history = interface.test_agent(num_episodes=20, verbose=1, seed=None, advanced_stats=True)

Testing progress:   0%|          | 0/20 [00:00<?, ?it/s]

Episode: 1/20, Score: -0.0047, 10 epi avg.: -0.0047, All-time avg.: -0.0047
Episode: 2/20, Score: 0.0099, 10 epi avg.: 0.0026, All-time avg.: 0.0026
Episode: 3/20, Score: 0.0130, 10 epi avg.: 0.0061, All-time avg.: 0.0061
Episode: 4/20, Score: 0.0033, 10 epi avg.: 0.0054, All-time avg.: 0.0054
Episode: 5/20, Score: -0.0066, 10 epi avg.: 0.0030, All-time avg.: 0.0030
Episode: 6/20, Score: -0.0016, 10 epi avg.: 0.0022, All-time avg.: 0.0022
Episode: 7/20, Score: -0.0011, 10 epi avg.: 0.0018, All-time avg.: 0.0018
Episode: 8/20, Score: 0.0027, 10 epi avg.: 0.0019, All-time avg.: 0.0019
Episode: 9/20, Score: -0.0066, 10 epi avg.: 0.0010, All-time avg.: 0.0010
Episode: 10/20, Score: -0.0019, 10 epi avg.: 0.0007, All-time avg.: 0.0007
Episode: 11/20, Score: 0.0090, 10 epi avg.: 0.0014, All-time avg.: 0.0014
Episode: 12/20, Score: -0.0028, 10 epi avg.: 0.0011, All-time avg.: 0.0011
Episode: 13/20, Score: -0.0020, 10 epi avg.: 0.0008, All-time avg.: 0.0008
Episode: 14/20, Score: -0.0039, 10 ep

### Render Last Episode

In [45]:
test_env.render(plot_benchmark=True, plot_actions=True, plot_relative=True, plot_returns=True, plot_individual=True, plot_contribution=True)

### Last Episode Performance

In [46]:
print('Agent Vol (Ann.): {:.3f}%'.format(np.std(test_env.portfolio_returns)*100*np.sqrt(252)))
print('Benchmark Vol (Ann.): {:.3f}%'.format(np.std(test_env.benchmark_returns)*100*(np.sqrt(252))))
print('Correlation: %.3f' % np.corrcoef(np.squeeze(test_env.portfolio_returns), np.squeeze(test_env.benchmark_returns))[0,1])
beta = np.cov(np.squeeze(test_env.portfolio_returns), np.squeeze(test_env.benchmark_returns))[0,1] / np.var(test_env.benchmark_returns)
print('Beta: %.3f' % beta)
print('Excess Return: {:.3f}%'.format((np.cumprod(1+test_env.portfolio_returns)[-1] - np.cumprod(1+test_env.benchmark_returns)[-1])*100))
print('Alpha (rf=0%): {:.3f}%'.format((np.cumprod(1+test_env.portfolio_returns)[-1] - beta*np.cumprod(1+test_env.benchmark_returns)[-1])*100))

Agent Vol (Ann.): 17.564%
Benchmark Vol (Ann.): 15.996%
Correlation: 0.920
Beta: 1.017
Excess Return: 12.158%
Alpha (rf=0%): 9.856%


### Testing Sample Performance Statistics

In [47]:
interface.print_advanced_stats(test_history)

Avg. Reward: 0.001
Avg. Volatility (Agent, Ann.): 23.409%
Avg. Risk Utilization (% of benchmark): 97.284%
Avg. Beta: 0.939
Avg. Correlation: 0.960
Avg. Return (Epi.): 138.028%
Avg. Excess Return (Epi.): 17.218%
Avg. Alpha (Ann.): -19.334%
